## Pandas ++
[referencia rapida](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_r.html#quick-reference)

In [ ]:
# vamos pensar em manipulação de dados fazendo um paralelo com o dplyr.
# dplyr: 6 verbos principais
# select()    # seleciona colunas do data.frame
# arrange()   # reordena as linhas do data.frame
# filter()    # filtra linhas do data.frame
# mutate()    # cria novas colunas no data.frame (ou atualiza as colunas existentes)
# summarise() + group_by() # sumariza o data.frame
# left_join   # junta dois data.frames

In [ ]:
import pandas as pd

imdb = pd.read_csv("../dados/imdb.csv")
imdb_simples = imdb.filter(["titulo", "ano"])


### Exercícios

Os exercícios abaixo consideram a base do IMDB.

In [ ]:
# Objetivo: descobrir qual o filme mais caro 

(imdb.
    query("ano >= 2000 & ano < 2010").
    query("orcamento == orcamento.max()")
)

In [ ]:
# Iloc is used to select some rows 
x = imdb.sort_values("orcamento",ascending=False).iloc(0)


In [ ]:
# Objetivo: pegar todos os filmes que sejam do genero "Comedy"
# One can only change a Series to a string, but not a Data Frame 
imdb_comedy = imdb[imdb["generos"].str.contains("Comedy")]


### Mutate

In [ ]:
imdb_simples.assign(
    coluna_nova = imdb_simples.filter(['ano']) < 1945,
    # não é muito recomendado fazer isso ^
    coluna_nova2 = imdb_simples.filter(['ano']) < 1945,
    # esse é o jeito elgal de criar essa coluna ^
    coluna_numero = imdb_simples.filter(['ano'])-1900
)

In [ ]:
(imdb.assign(
    lucro = lambda x: x.receita-x.orcamento,
    # essa linha é igual a trocar o x. por imdb. e tirar o "lambda x"
    # x. vira imdb., então:  x.receita-x.orcamento vira imdb.receita-imdb.orcamento
    categoria_lucrou = lambda x: x.lucro > 0,
    duracao = imdb.duracao/60
).
 filter(["titulo","ano", "lucro", "duracao", "categoria_lucrou"]))

In [ ]:
imdb2 = imdb.copy()
imdb2['lucrou'] = imdb2['receita'] > imdb2['orcamento']
imdb2.filter(["lucrou"])

In [ ]:
imdb = imdb.drop(["lucrou"], axis = "columns")

In [ ]:
imdb2 = imdb.copy()
imdb2['lucrou'] = imdb2['receita'] > imdb2['orcamento']
imdb2.filter(["lucrou"])

In [ ]:
# case when
import numpy as np
lucrou_df = imdb.assign(
    lucro = lambda x: x["receita"] - x['orcamento'],
    lucrou_bastante = lambda x: np.select(
        [x.lucro >  1000000, x.lucro > 0, x.lucro.isnull()], 
        ["Mais de milhão", "Pouco", "Sem info"], 
        "Não lucrou"
    )
).filter(["lucro", "lucrou_bastante"])

In [ ]:
lucrou_df.lucrou_bastante.value_counts()

### Exercícios

In [ ]:
# Obter um dataframe com os filmes que deram prejuizo.


In [ ]:
# Obter os filmes que possuem a palavra 'love' no título.
(imdb.assign(
    imdb_love = lambda x: x.titulo.str.contains("love")
    ).
    query("imdb_love")
)

In [26]:
# Obter os files com gênero 'Drama' e ordernar pela quantidade de
# vezes que a palavra 'woman' (mulher) aparece na descrição.
df= (imdb.assign(
    imdb_drama = lambda x: x.generos.str.contains("Drama"),
    imdb_women = lambda x: x.descricao.str.lower().str.count("women")
    ).
    query("imdb_drama").
    sort_values( ['imdb_women'], ascending=False )
)



### Summarise
###### 4 options to aggregate

In [27]:
imdb.filter(["duracao", "orcamento"]).agg(["mean", "sum", "count"])

,duracao,orcamento
mean,9.305483e+01,1.213561e+07
sum,2.651132e+06,1.270598e+11
count,2.849000e+04,1.047000e+04


In [30]:
  imdb.agg({
  "duracao": ["mean"],
  "orcamento": ["mean", "count"]
})

,duracao,orcamento
mean,93.054826,1.213561e+07
count,NaN,1.047000e+04


In [37]:
imdb.groupby(["ano"], as_index = False).agg(
  duracao_media = ("duracao", "mean"),
  desv_pad_duracao = ("duracao", "std"),
  receita_max = ("receita", max)
)


,ano,duracao_media,desv_pad_duracao,receita_max
0,1894.0,45.000000,NaN,NaN
1,1912.0,80.000000,28.284271,NaN
2,1913.0,88.000000,NaN,NaN
3,1914.0,73.312500,35.254728,NaN
4,1915.0,74.066667,34.900812,NaN
...,...,...,...,...
105,2016.0,93.359033,13.625927,1.153332e+09
106,2017.0,92.982320,13.029991,1.332540e+09
107,2018.0,93.658014,13.717957,2.048360e+09
108,2019.0,95.173104,18.143177,2.797801e+09


In [36]:
imdb.groupby('ano', as_index=False).apply(lambda x: pd.DataFrame({
    "duracao_media": [x.duracao.mean()],
    "desv_pad_duracao": [x.duracao.std()],
    "receita_max": [x.receita.max()],
    "lucro_medio": [(x.receita - x.orcamento).mean()]
}))

,,duracao_media,desv_pad_duracao,receita_max,lucro_medio
0,0,45.000000,NaN,NaN,NaN
1,0,80.000000,28.284271,NaN,NaN
2,0,88.000000,NaN,NaN,NaN
3,0,73.312500,35.254728,NaN,NaN
4,0,74.066667,34.900812,NaN,NaN
...,...,...,...,...,...
105,0,93.359033,13.625927,1.153332e+09,7.237195e+07
106,0,92.982320,13.029991,1.332540e+09,9.284472e+07
107,0,93.658014,13.717957,2.048360e+09,1.122318e+08
108,0,95.173104,18.143177,2.797801e+09,1.275660e+08


### Exercícios

In [ ]:
# Obtenha os 10 anos com as as maiores notas médias do IMDB.
# Coloque em um data frame ordeno pela média das notas.

In [ ]:
# Obtenha a proporção de filmes que deram prejuizo por ano.

### Left join

In [ ]:
lucro_direcao = (imdb.
    assign(
        lucro = lambda x: x.receita - x.orcamento
    ).
    groupby(["direcao"], as_index = False).
    agg(
        lucro_medio = ('lucro', 'mean')
    )
)
lucro_direcao

In [ ]:
imdb_com_lucro = pd.merge(imdb, lucro_direcao, how = "left", on = ["direcao"])

In [ ]:
(imdb_com_lucro.
    assign(lucro = lambda x: x.receita - x.orcamento).
    filter(["titulo", "lucro", "lucro_medio"])
)